In [ ]:
import os, cv2, random, json
import numpy as np
import matplotlib.pyplot as plt
import torch

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image

### I.

In [ ]:
input_path = "./dataset"

train_path = os.path.join(input_path, "train")
val_path = os.path.join(input_path, "val")
test_path = os.path.join(input_path, "test")

data_dir = [dir for dir in sorted(os.listdir(input_path)) if os.path.isdir(os.path.join(input_path, dir))]
label_names = [subdir for subdir in sorted(os.listdir(train_path)) if os.path.isdir(os.path.join(train_path, subdir))]

In [ ]:
paths = [train_path, val_path, test_path]
split_names = ["train", "val", "test"]

num_splits = len(paths)
num_classes = len(label_names)

fig, axes = plt.subplots(num_classes, num_splits, figsize=(15, 10))

for i in range(num_splits):
    for j in range(num_classes):
        current_folder = os.path.join(paths[i], label_names[j])

        all_images = [f for f in os.listdir(current_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if all_images:
            img_name = random.choice(all_images)
            img_path = os.path.join(current_folder, img_name)

            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            ax = axes[j, i]; ax.imshow(img)
            ax.set_title(f"{split_names[i]}: {label_names[j]}", fontsize=10); ax.axis('off')

plt.show()

### II.

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.GaussNoise(std_range=(0.012, 0.027), p=0.2),
    A.ISONoise(color_shift=(0.15, 0.35), intensity=(0.1, 0.5), p=0.05),
    A.ImageCompression(quality_range=(50, 100), p=0.25),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.Affine(scale=(0.9, 1.1), translate_percent=(0.1, 0.1), rotate=(-15, 15), interpolation=cv2.INTER_CUBIC, border_mode=cv2.BORDER_REFLECT_101, p=0.5),
    A.CoarseDropout(num_holes_range=(4, 8), hole_height_range=(4, 16), hole_width_range=(4, 16), fill=0, p=0.25),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_test_transforms = A.Compose([
    A.Resize(224, 224, interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

### III.

In [ ]:
import pandas as pd
import os

def gen_df(img_dir, labels):
    real_img_dir = os.path.join(img_dir, labels[0])
    spoof_img_dir = os.path.join(img_dir, labels[1])

    real_files = [os.path.join(real_img_dir, f) for f in os.listdir(real_img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    spoof_files = [os.path.join(spoof_img_dir, f) for f in os.listdir(spoof_img_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    data = {
        'file_path': real_files + spoof_files,
        'label': [1] * len(real_files) + [0] * len(spoof_files)
    }

    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df

df_train = gen_df(train_path, label_names)
df_val   = gen_df(val_path, label_names)
df_test  = gen_df(test_path, label_names)

In [ ]:
df_train_0 = df_train[df_train['label']==0][:1223]
df_train_1 = df_train[df_train['label']==1][:1223]
df_train_balanced = pd.concat([df_train_0, df_train_1]).reset_index(drop=True)
df_train_balanced = df_train_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_val_0 = df_val[df_val['label']==0][:405]
df_val_1 = df_val[df_val['label']==1][:405]
df_val_balanced = pd.concat([df_val_0, df_val_1]).reset_index(drop=True)
df_val_balanced = df_val_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
df_test_0 = df_test[df_test['label']==0][:314]
df_test_1 = df_test[df_test['label']==1][:314]
df_test_balanced = pd.concat([df_test_0, df_test_1]).reset_index(drop=True)
df_test_balanced = df_test_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
class FASDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['file_path']
        label = self.df.iloc[idx]['label']

        image = Image.open(img_path).convert("RGB")
        image = np.array(image)

        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented['image']

        return image, torch.tensor(label, dtype=torch.float32).unsqueeze(0)

In [ ]:
train_dataset = FASDataset(df_train_balanced, train_transforms)
val_dataset = FASDataset(df_val_balanced, val_test_transforms)
test_dataset = FASDataset(df_test_balanced, val_test_transforms)

dataloader_train = DataLoader(train_dataset, batch_size=32, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=32, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=32, shuffle=True)

### IV.

In [ ]:
import torch.nn as nn
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights

class SpoofNet(nn.Module):
    def __init__(self):
        super(SpoofNet, self).__init__()
        weights = MobileNet_V2_Weights.DEFAULT
        self.backbone = mobilenet_v2(weights=weights).features

        self.custom_layers = nn.Sequential(
            nn.Conv2d(1280, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.custom_layers(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpoofNet().to(device)

In [ ]:
# import torch.optim as optim
# from torch.optim.lr_scheduler import ReduceLROnPlateau
#
# num_epochs = 30
# learning_rate = 5e-5
#
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.BCELoss()
# scheduler = ReduceLROnPlateau(
#     optimizer, factor=0.2, patience=3, mode='min',
#     threshold=0.005, min_lr=5e-7,
# )
#
# history = {
#     'train_loss': [], 'val_loss': [],
#     'train_accuracy': [], 'val_accuracy': [],
#     'learning_rate': [],
# }
#
# best_val_loss = float('inf')
# save_dir = './models'
#
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
#
# best_filepath = os.path.join(save_dir, "mobilenetv2-best.pt")
#
# def save_checkpoint(state, is_best):
#     if is_best:
#         torch.save(state, best_filepath)

In [ ]:
# for epoch in range(num_epochs):
#     model.train()
#     train_metrics = {'loss': 0.0, 'correct': 0, 'total': 0}
#
#     pbar_train = tqdm(dataloader_train, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
#     for inputs, labels in pbar_train:
#         inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
#
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#
#         train_metrics['loss'] += loss.item()
#         preds = (outputs > 0.5).float()
#         train_metrics['correct'] += (preds == labels).sum().item()
#         train_metrics['total'] += labels.size(0)
#
#         pbar_train.set_postfix(acc=f"{100 * train_metrics['correct'] / train_metrics['total']:.2f}%")
#
#     model.eval()
#     val_metrics = {'loss': 0.0, 'correct': 0, 'total': 0}
#
#     pbar_val = tqdm(dataloader_val, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)
#     with torch.no_grad():
#         for inputs, labels in pbar_val:
#             inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
#             outputs = model(inputs)
#
#             val_metrics['loss'] += criterion(outputs, labels).item()
#             val_metrics['correct'] += ((outputs > 0.5).float() == labels).sum().item()
#             val_metrics['total'] += labels.size(0)
#
#     avg_train_loss = train_metrics['loss'] / len(dataloader_train)
#     avg_val_loss = val_metrics['loss'] / len(dataloader_val)
#     train_acc = 100 * train_metrics['correct'] / train_metrics['total']
#     val_acc = 100 * val_metrics['correct'] / val_metrics['total']
#     current_lr = optimizer.param_groups[0]['lr']
#
#     scheduler.step(avg_val_loss)
#
#     for k, v in zip(history.keys(), [avg_train_loss, avg_val_loss, train_acc, val_acc, current_lr]):
#         history[k].append(v)
#
#     is_best = avg_val_loss < best_val_loss
#     if is_best:
#         best_val_loss = avg_val_loss
#
#         save_checkpoint({
#             'epoch': epoch + 1,
#             'state_dict': model.state_dict(),
#             'optimizer': optimizer.state_dict(),
#         }, is_best)
#
#     print(f"\nEpoch {epoch+1}: Train Loss {avg_train_loss:.4f}, Acc {train_acc:.2f}% | Val Loss {avg_val_loss:.4f}, Acc {val_acc:.2f}% | LR: {current_lr}")
#
# history_path = os.path.join(save_dir, 'training_history.json')
# with open(history_path, 'w') as f:
#     json.dump(history, f)

### V.

In [ ]:
with open('training_history.json', 'r') as f:
    history = json.load(f)

epochs = range(1, len(history['train_loss']) + 1)

plt.style.use('seaborn-v0_8-whitegrid')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

ax1.plot(epochs, history['train_loss'], 'b-o', label='Train Loss', markersize=4)
ax1.plot(epochs, history['val_loss'], 'r-s', label='Val Loss', markersize=4)
ax1.set_title('Model Loss (Binary Cross Entropy)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.legend()

ax2.plot(epochs, history['train_accuracy'], 'b-o', label='Train Accuracy', markersize=4)
ax2.plot(epochs, history['val_accuracy'], 'r-s', label='Val Accuracy', markersize=4)
ax2.set_title('Model Accuracy (%)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.legend()

plt.tight_layout()
plt.savefig('training_report.png', dpi=300); plt.show()

In [ ]:
pretrain_weight = 'mobilenetv2-best.pt'

check_point = torch.load(pretrain_weight, map_location=torch.device('cpu'))

model_dict = check_point['state_dict']
epoch_ = check_point['epoch']

model.load_state_dict(model_dict)

model.to(device)
model.eval()
criterion = nn.BCELoss()

In [ ]:
test_loss = 0.0
correct_predictions = 0
total_predictions = 0
prog_bar_test = tqdm(dataloader_test, desc='Testing')

with torch.no_grad():
    for inputs, labels in prog_bar_test:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        test_loss += loss.item()

        predicted = (outputs > 0.5).int()

        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        prog_bar_test.set_postfix({
            'accuracy': correct_predictions / total_predictions * 100,
        })

test_loss /= len(dataloader_test)
accuracy = correct_predictions / total_predictions * 100

print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {accuracy:.2f}%')